In [15]:
# from aerobot.plot import plot_training_curve
import os
from aerobot.io import load_results_dict, RESULTS_PATH
import matplotlib.pyplot as plt
from aerobot.plot import plot_configure_mpl
from matplotlib.gridspec import GridSpec
from aerobot.dataset import dataset_load_training_validation
import pandas as pd 
import numpy as np 

%load_ext autoreload
%autoreload 2

plot_configure_mpl(n_colors=3)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# SI Figure 1

The first supplementary figure consists of two panels.

- Panel A: The phylogenetic composition of the training and validation datasets at the phylum level. Each phylum is represented by a split bar, where each portion of the bar represents the number of entries in the training or validation dataset belonging to that phylum.
- Panel B: The labels of entries in the training and validation datasets, i.e. aerobe, anaerobe, or facultative. This should also be represented as a stacked bar plot. 


In [16]:
training_dataset, validation_dataset = dataset_load_training_validation(feature_type='KO', to_numpy=False) # Feature type doesn't matter, as we just want the labels. 
training_labels, validation_labels = training_dataset['labels'], validation_dataset['labels']

In [17]:
training_counts = training_labels['Phylum'].value_counts().to_dict()
validation_counts = validation_labels['Phylum'].value_counts().to_dict()
phyla = list(set(validation_counts.keys()).union(set(training_counts.keys())))

phyla_counts_df = pd.DataFrame()
phyla_counts_df['phylum'] = phyla
phyla_counts_df['training'] = [training_counts.get(phylum, 0) for phylum in phyla]
phyla_counts_df['validation'] = [validation_counts.get(phylum, 0) for phylum in phyla]
phyla_counts_df['total'] = phyla_counts_df['validation'] + phyla_counts_df['training']
phyla_counts_df = phyla_counts_df.sort_values('total', ascending=False)

# Only look at the top 50 most abundant phyla...
phyla_counts_df = phyla_counts_df.iloc[:20]
# phyla_counts_df.head()

In [18]:
training_counts = training_labels['physiology'].value_counts().to_dict()
validation_counts = validation_labels['physiology'].value_counts().to_dict()
labels = ['aerobe', 'anaerobe', 'facultative']

labels_counts_df = pd.DataFrame()
labels_counts_df['labels'] = labels
labels_counts_df['training'] = [training_counts.get(label, 0) for label in labels]
labels_counts_df['training'] = labels_counts_df.training / labels_counts_df.training.sum() # Convert to fractions. 
labels_counts_df['validation'] = [validation_counts.get(label, 0) for label in labels]
labels_counts_df['validation'] = labels_counts_df.validation / labels_counts_df.validation.sum() # Convert to fractions. 

labels_counts_df = labels_counts_df.set_index('labels')
labels_counts_df


,training,validation
labels,,
aerobe,0.505849,0.514416
anaerobe,0.360865,0.368741
facultative,0.133286,0.116844


In [19]:
def plot_si_figure_1a(phyla_counts_df, grid:GridSpec=None, fig:plt.Figure=None):

    ax = fig.add_subplot(grid)
    n_phyla = len(phyla_counts_df)

    ax.bar(np.arange(n_phyla), phyla_counts_df.training.values)
    ax.bar(np.arange(n_phyla), phyla_counts_df.validation.values, bottom=phyla_counts_df.training.values)

    ax.set_xticks(np.arange(n_phyla), labels=phyla_counts_df.phylum, rotation=90)
    ax.legend(['training data', 'validation data'])
    ax.set_ylabel('number of genomes')

    ax.set_title('Phylogenetic coverage')


In [22]:
def plot_si_figure_1b(labels_counts_df, grid:GridSpec=None, fig:plt.Figure=None):

    ax = fig.add_subplot(grid)

    ax.bar(np.arange(2), labels_counts_df.iloc[0].values)
    ax.bar(np.arange(2), labels_counts_df.iloc[1].values, bottom=labels_counts_df.iloc[0].values)
    ax.bar(np.arange(2), labels_counts_df.iloc[2].values, bottom=labels_counts_df.iloc[0:2].values.sum(axis=0))

    ax.set_xticks(np.arange(2), labels=['training', 'validation'])
    ax.legend(['aerobe', 'anaerobe', 'facultative'])
    ax.set_ylabel('proportion')


In [23]:
fig = plt.figure(figsize=(10, 3))
grid = GridSpec(1, 2, wspace=0.2) 

plot_si_figure_1a(phyla_counts_df, grid=grid[0, 0], fig=fig)
plot_si_figure_1b(labels_counts_df, grid=grid[0, 1], fig=fig)
